# 变量定义

In [1]:
import os

ROOT_DATA = os.environ.get('LARD_DATA_ROOT_PATH')
ROOT_PROJECT = os.environ.get('LARD_PROJECT_ROOT_PATH')

print(ROOT_DATA)
print(ROOT_PROJECT)


/fileonssd/runway-dataset/lard-dataset
/home/yeli/workspace/lard/lard-detection


In [2]:
path_best_model = f'{ROOT_PROJECT}/runs/ultralytics/finetune/yolov8n_lska_bifpn_aug_color_blur_geo_weather_10_640/triple/train/weights/last.pt'
path_video = f'{ROOT_PROJECT}/datasets/video/landing_1.mp4'

print(f'{path_best_model}')
print(f'{path_video}')

/home/yeli/workspace/lard/lard-detection/runs/ultralytics/finetune/yolov8n_lska_bifpn_aug_color_blur_geo_weather_10_640/triple/train/weights/last.pt
/home/yeli/workspace/lard/lard-detection/datasets/video/landing_1.mp4


# 获取视频信息

In [5]:
import cv2

# 原始参数
cap = cv2.VideoCapture(path_video)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

# 输出视频参数
ratio_compression = 1
frame_width = int(frame_width/ratio_compression)
frame_height = int(frame_height/ratio_compression)

# 打印参数
print(f"frame_width: {frame_width}")
print(f"frame_height: {frame_height}")
print(f"frame_rate: {frame_rate}")
print(f"total_frames: {total_frames}")

frame_width: 1920
frame_height: 1008
frame_rate: 30
total_frames: 241


In [4]:
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

# 视频 - 检测

In [6]:
import cv2
from tqdm import tqdm
from ultralytics import YOLO

# 加载模型
model = YOLO(path_best_model)

# 打开原始视频
cap = cv2.VideoCapture(path_video)

# 创建VideoWriter对象
path_output = f'{ROOT_PROJECT}/results/videos/output-detection.mp4'
out = cv2.VideoWriter(path_output, fourcc, frame_rate, (frame_width, frame_height))

# 初始化进度条
with tqdm(total=total_frames, desc="处理进度", ncols=100) as pbar:
    # 遍历视频
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            results = model.predict(frame, verbose=False)
            frame = results[0].plot()
            frame = cv2.resize(frame, (frame_width, frame_height))
            out.write(frame)
            pbar.update(1)
        else:
            break

# 释放资源
cap.release()
out.release()

# 文件尺寸计算
file_size = os.path.getsize(path_output) / (1024 ** 2)
print(f"文件保存至: {path_output} 文件大小: {file_size:.2f} MB")

处理进度: 100%|███████████████████████████████████████████████████| 241/241 [00:06<00:00, 38.19it/s]

文件保存至: /home/yeli/workspace/lard/lard-detection/results/videos/output-detection.mp4 文件大小: 15.49 MB


# 视频 - 追踪

In [7]:
import cv2
from tqdm import tqdm

# 加载模型
model = YOLO(path_best_model)

# 打开原始视频
cap = cv2.VideoCapture(path_video)

# 创建VideoWriter对象
path_output = f'{ROOT_PROJECT}/results/videos/output-track.mp4'
out = cv2.VideoWriter(path_output, fourcc, frame_rate, (frame_width, frame_height))

# 初始化进度条
with tqdm(total=total_frames, desc="处理进度", ncols=100) as pbar:
    # 遍历视频
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            results = model.track(frame, persist=True, verbose=False)
            frame = results[0].plot()
            frame = cv2.resize(frame, (frame_width, frame_height))
            out.write(frame)
            pbar.update(1)
        else:
            break

# 释放资源
cap.release()
out.release()

# 文件尺寸计算
file_size = os.path.getsize(path_output) / (1024 ** 2)
print(f"文件保存至: {path_output} 文件大小: {file_size:.2f} MB")

处理进度: 100%|███████████████████████████████████████████████████| 241/241 [00:07<00:00, 33.07it/s]

文件保存至: /home/yeli/workspace/lard/lard-detection/results/videos/output-track.mp4 文件大小: 15.56 MB


# 视频-追踪-光流

In [8]:
from collections import defaultdict
from tqdm import tqdm
import cv2
import numpy as np

# 加载模型
model = YOLO(path_best_model)

# 打开原始视频
cap = cv2.VideoCapture(path_video)

# 创建VideoWriter对象
path_output = f'{ROOT_PROJECT}/results/videos/output-track-path.mp4'
out = cv2.VideoWriter(path_output, fourcc, frame_rate, (frame_width, frame_height))

# 存储追踪历史
track_history = defaultdict(lambda: [])

# 初始化进度条
with tqdm(total=total_frames, desc="处理进度", ncols=100) as pbar:
    # 遍历视频
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            results = model.track(frame, persist=True, verbose=False)
            
            # 如果检测到并追踪到目标
            if results[0].boxes.xywh is not None and results[0].boxes.id is not None:

                # 获取boxes和追踪track IDs
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()

                # 带有追踪结果的图像
                frame = results[0].plot()

                # 绘制轨迹
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]  # track_id的历史轨迹
                    # 追加新跟踪到的点 x, y center point
                    track.append((float(x), float(y)))
                    if len(track) > 30:  # 为每个目标保留30个历史轨迹
                        track.pop(0)

                    # 画出轨迹线
                    points = np.hstack(track).astype(
                        np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [points], isClosed=False, color=(0, 255, 255), thickness=10)
                frame = cv2.resize(frame, (frame_width, frame_height))
                out.write(frame)

            # 检测到但没有追踪到目标
            elif results[0].boxes.xywh is not None:
                frame = results[0].plot()
                frame = cv2.resize(frame, (frame_width, frame_height))
                out.write(frame)

            # 没有检测到也没有追踪到
            else:
                frame = cv2.resize(
                    frame, (frame_width, frame_height))
                out.write(frame)
            pbar.update(1)
        else:
            break

# 释放资源
cap.release()
out.release()

# 文件尺寸计算
file_size = os.path.getsize(path_output) / (1024 ** 2)
print(f"文件保存至: {path_output} 文件大小: {file_size:.2f} MB")

处理进度: 100%|███████████████████████████████████████████████████| 241/241 [00:07<00:00, 30.57it/s]

文件保存至: /home/yeli/workspace/lard/lard-detection/results/videos/output-track-path.mp4 文件大小: 15.58 MB
